In [3]:
import findspark

findspark.init()
 

In [4]:
# !pip install findspark

In [5]:
from pyspark import SparkContext

In [6]:
sc = SparkContext(master="local[2]")

In [7]:
import pyspark

from pyspark.sql import SparkSession

# spark = SparkSession.builder.getOrCreate()

# df = spark.sql("select 'spark' as hello ")

# df.show()

In [8]:
#sc = spark

In [9]:
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [10]:
spark = SparkSession.builder.appName("TextClassifierApp").getOrCreate()

In [48]:
path = r"C:\Users\gprak\Downloads\projects\Data\86b13d9a4b8e11ec\project\training_data.csv"
df = spark.read.csv(path,header=True,inferSchema=True)


In [49]:
df.show()

+--------------------+--------------------+
|               title|            category|
+--------------------+--------------------+
|    The Three Amigos|                None|
|Home Essentials B...|      Home & Kitchen|
|Cooper Wiring Qui...|Tools & Home Impr...|
|Baseboarders&reg;...|Tools & Home Impr...|
|The Great Wave Of...|     Office Products|
|Nemcor Pittsburgh...|      Home & Kitchen|
|Patrician Berkley...|                None|
|SouvNear 81461402...|                None|
|20 Qty. Halco 50W...|Tools & Home Impr...|
|      Rilakkuma Bowl|                None|
|Redbirdlinen 1pc ...|      Home & Kitchen|
|Hospital Bath Tow...|      Home & Kitchen|
|Symphony in Red a...|      Home & Kitchen|
|Big Train BLENDED...|Grocery & Gourmet...|
|Surpahs Round 11 ...|      Home & Kitchen|
|Mikasa Love Story...|                None|
|180 Snacks Nutty ...|                None|
|Anti-Slip Handle ...|      Home & Kitchen|
|Imagine Thicket G...|                None|
|KOHLER K-3754-96 ...|          

In [50]:
df.columns

['title', 'category']

In [51]:
#df=df.dropna()

In [52]:
df.columns

['title', 'category']

In [53]:
import pyspark.ml.feature

In [54]:
dir(pyspark.ml.feature)

['Binarizer',
 'BucketedRandomProjectionLSH',
 'BucketedRandomProjectionLSHModel',
 'Bucketizer',
 'ChiSqSelector',
 'ChiSqSelectorModel',
 'CountVectorizer',
 'CountVectorizerModel',
 'DCT',
 'ElementwiseProduct',
 'FeatureHasher',
 'HasFeaturesCol',
 'HasHandleInvalid',
 'HasInputCol',
 'HasInputCols',
 'HasLabelCol',
 'HasMaxIter',
 'HasNumFeatures',
 'HasOutputCol',
 'HasOutputCols',
 'HasRelativeError',
 'HasSeed',
 'HasStepSize',
 'HasThreshold',
 'HasThresholds',
 'HashingTF',
 'IDF',
 'IDFModel',
 'Imputer',
 'ImputerModel',
 'IndexToString',
 'Interaction',
 'JavaEstimator',
 'JavaMLReadable',
 'JavaMLWritable',
 'JavaModel',
 'JavaParams',
 'JavaTransformer',
 'MaxAbsScaler',
 'MaxAbsScalerModel',
 'MinHashLSH',
 'MinHashLSHModel',
 'MinMaxScaler',
 'MinMaxScalerModel',
 'NGram',
 'Normalizer',
 'OneHotEncoder',
 'OneHotEncoderModel',
 'PCA',
 'PCAModel',
 'Param',
 'Params',
 'PolynomialExpansion',
 'QuantileDiscretizer',
 'RFormula',
 'RFormulaModel',
 'RegexTokenizer',
 'R

In [55]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF

In [56]:
from pyspark.ml.feature import StringIndexer

In [57]:
tokenizer = Tokenizer(inputCol='title',outputCol='mytokens')
stopwords_remover = StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [58]:
labelEncoder = StringIndexer(inputCol='category',outputCol='label').fit(df)

In [59]:
labelEncoder.transform(df).show(5)

+--------------------+--------------------+-----+
|               title|            category|label|
+--------------------+--------------------+-----+
|    The Three Amigos|                None|  0.0|
|Home Essentials B...|      Home & Kitchen|  1.0|
|Cooper Wiring Qui...|Tools & Home Impr...|  2.0|
|Baseboarders&reg;...|Tools & Home Impr...|  2.0|
|The Great Wave Of...|     Office Products|  3.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [60]:
# label_dict = {'Web Development':0.0,
#  'Business Finance':1.0,
#  'Musical Instruments':2.0,
#  'Graphic Design':3.0}

In [61]:
df = labelEncoder.transform(df)

In [62]:
df.show(5)

+--------------------+--------------------+-----+
|               title|            category|label|
+--------------------+--------------------+-----+
|    The Three Amigos|                None|  0.0|
|Home Essentials B...|      Home & Kitchen|  1.0|
|Cooper Wiring Qui...|Tools & Home Impr...|  2.0|
|Baseboarders&reg;...|Tools & Home Impr...|  2.0|
|The Great Wave Of...|     Office Products|  3.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [71]:

df = df.filter("category != 'None'")

In [72]:
df.show(5)

+--------------------+--------------------+-----+
|               title|            category|label|
+--------------------+--------------------+-----+
|Home Essentials B...|      Home & Kitchen|  1.0|
|Cooper Wiring Qui...|Tools & Home Impr...|  2.0|
|Baseboarders&reg;...|Tools & Home Impr...|  2.0|
|The Great Wave Of...|     Office Products|  3.0|
|Nemcor Pittsburgh...|      Home & Kitchen|  1.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [75]:
df.count()

10067

In [76]:
(trainDF,testDF) = df.randomSplit((0.7,0.3),seed=42)

In [77]:
from pyspark.ml.classification import LogisticRegression

In [78]:
lr = LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')

In [79]:
from pyspark.ml import Pipeline 

In [80]:
pipeline = Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,lr])

In [81]:
lr_model = pipeline.fit(trainDF)

In [82]:
predictions = lr_model.transform(testDF)

In [83]:
predictions.show()

+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|            category|label|            mytokens|     filtered_tokens|         rawFeatures|  vectorizedFeatures|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| BAZIC Sports Pus...|     Office Products|  3.0|[, bazic, sports,...|[, bazic, sports,...|(22831,[13,37,46,...|(22831,[13,37,46,...|[-3.6123809322418...|[8.02262906045998...|       3.0|
| Biedermann &amp;...|      Home & Kitchen|  1.0|[, biedermann, &a...|[, biedermann, &a...|(22831,[5,28,36,4...|(22831,[5,28,36,4...|[-3.6177162823991...|[2.14357793084081...|       1.0|
| Bosco Chocolate ...|Grocery & Gourmet...|  4.0|[, bosco, chocol

In [84]:
predictions.columns

['title',
 'category',
 'label',
 'mytokens',
 'filtered_tokens',
 'rawFeatures',
 'vectorizedFeatures',
 'rawPrediction',
 'probability',
 'prediction']

In [85]:
predictions.select('rawPrediction','probability','category','label','prediction').show(10)

+--------------------+--------------------+--------------------+-----+----------+
|       rawPrediction|         probability|            category|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|[-3.6123809322418...|[8.02262906045998...|     Office Products|  3.0|       3.0|
|[-3.6177162823991...|[2.14357793084081...|      Home & Kitchen|  1.0|       1.0|
|[-3.6166644152096...|[1.86825934222308...|Grocery & Gourmet...|  4.0|       4.0|
|[-3.6023739291447...|[3.62586254494038...|Grocery & Gourmet...|  4.0|       4.0|
|[-3.6225852870114...|[2.18134258266067...|      Home & Kitchen|  1.0|       1.0|
|[-3.6288732627192...|[2.65979726232791...|      Home & Kitchen|  1.0|       1.0|
|[-3.6225072507470...|[6.76621847742010...|      Home & Kitchen|  1.0|       1.0|
|[-3.6173807476387...|[7.11279118260594...|      Home & Kitchen|  1.0|       1.0|
|[-3.6257410157263...|[6.26283258921556...|     Office Products|  3.0|       3.0|
|[-3.61722693916

In [86]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [87]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')

In [88]:
accuracy = evaluator.evaluate(predictions)

In [89]:
accuracy

0.7675694206376414

# Fine tuning

In [90]:
# import pyspark.ml.tuning as tune

# # Create the parameter grid
# grid = tune.ParamGridBuilder()

# # Add the hyperparameter
# grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
# grid = grid.addGrid(lr.elasticNetParam, [0, 1])

# # Build the grid
# grid = grid.build()

In [91]:
df.columns

['title', 'category', 'label']

In [92]:
# Create the CrossValidator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
# cv = tune.CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator)

In [95]:


# #training_data = trainDF
# #testing_data = testDF

# paramGrid = ParamGridBuilder().addGrid(lr.regParam,[0.02,0.08])\
#             .addGrid(lr.elasticNetParam,[0.2,0.6]).build()

# #evaluator_lr = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
# evaluator_lr = evaluator
# crossval = CrossValidator(estimator=lr,
#                           estimatorParamMaps=paramGrid,
#                           evaluator=evaluator_lr,
#                           numFolds=2)

# #start_time = time()
# pipeline2 = Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf])
# piped_data = pipeline2.fit(df).transform(df)

# # Split the data into training and test sets
# training, test = piped_data.randomSplit([.7, .3])

# #best_lr = lr.fit(training)

# # # Print best_lr
# #print(best_lr)

# # Run cross-validation, and choose the best set of parameters.
# cvModel = crossval.fit(training)

# #end_time = time()

# #training_time = end_time - start_time

# #print("The time taken to train the data is: %0.3f seconds" %training_time)

# # Make predictions on testing data and calculating ROC metrics and model accuracy. 
# prediction = cvModel.transform(test)
# # #output= prediction.select("features",  "probability", "prediction")
# # output = prediction
# # acc = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "accuracy"})
# # f1 = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "f1"})
# # weightedPrecision = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "weightedPrecision"})
# # weightedRecall = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "weightedRecall"})
# # auc = evaluator_lr.evaluate(output)

# # print(acc)
# # print(f1)
# # print(weightedPrecision)
# # print(weightedRecall)
# # print(auc)

In [96]:
# output = prediction
# acc = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "accuracy"})
# f1 = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "f1"})
# weightedPrecision = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "weightedPrecision"})
# weightedRecall = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "weightedRecall"})
# auc = evaluator_lr.evaluate(output)

# print(acc)
# print(f1)
# print(weightedPrecision)
# print(weightedRecall)
# print(auc)

# Step 3. Create a Multi Layer Perceptron

In [140]:
# from pyspark.ml.classification import MultilayerPerceptronClassifier
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
# from pyspark.ml import Pipeline
# from pyspark.sql.functions import udf, StringType
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# from pyspark.ml.classification import MultilayerPerceptronClassifier

In [141]:
# data=df
# print(df.count())
# train, validation, test = data.randomSplit([0.7, 0.1, 0.3], 1234)

In [142]:
# categorical_columns = [item[0] for item in data.dtypes if item[1].startswith(
#     'string')]
# numeric_columns = [item[0] for item in data.dtypes if item[1].startswith(
#     'double')]
# indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(
#     column)) for column in categorical_columns]

In [143]:
# data.dtypes

In [144]:
# [indexer.getOutputCol() for indexer in indexers] 

In [145]:
# featuresCreator = VectorAssembler(
#     inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns,
#     outputCol='features')
# layers = [len(featuresCreator.getInputCols()), 40, 20, 12]

In [146]:
# print(layers)

In [147]:
# classifier = MultilayerPerceptronClassifier(labelCol='label',
#                                             featuresCol='features',
#                                             maxIter=100,
#                                             layers=layers,
#                                             blockSize=128,
#                                             seed=1234)

In [148]:
# pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])
# model = pipeline.fit(train)


In [149]:
# train_output_df = model.transform(train)
# validation_output_df = model.transform(validation)
# test_output_df = model.transform(test)

In [150]:
# train_predictionAndLabels = train_output_df.select('prediction', 'label')
# validation_predictionAndLabels = validation_output_df.select('prediction', 'label')
# test_predictionAndLabels = test_output_df.select('prediction', 'label')

In [151]:
# metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']
# for metric in metrics:
#     evaluator = MulticlassClassificationEvaluator(metricName=metric)
#     print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
#     # print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
#     # pr/int('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))